# Setting Up
- Set your working directory where the ImageJ Results file CSVs are stored in `dir`. Here we assume that they are in a subfolder named `data`.

In [12]:
import os
cwd = os. getcwd()
dir = f'{cwd}/data'

# Functions

In [4]:
%matplotlib inline
import  pandas as pd, numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pylab as pl

plt.style.use('default')
plt.rcParams['font.size'] = 10 
plt.rcParams['axes.linewidth'] = 1.6
plt.rcParams["font.family"] = "Arial"

def readMultiMeasureData(path):
    """function that reads in ImageJ Multiple Measure data from a folder and returns a Pandas dataframe. """
    df = pd.read_csv(dir+path)
    df = df.drop(df.columns[0],1)
    df.columns = list(range(1,len(df.columns)+1))
    return df

def subtractBackground(df,bg):
    for i in df.columns:
        df[i] = df[i] - bg.mean(axis=1) 
    return df

def plotTraces(df,yaxis='Intensity [a.u.]'):
    """helper function to quickly visualize individual spore timetraces"""
    vs = np.random.randn(len(df.columns))
    normal = matplotlib.colors.Normalize(vs.min(), vs.max())
    colors = pl.cm.jet(normal(vs))
    
    ax = df.plot(lw=1.5,color=colors,alpha=0.1)
    ax.legend(loc='upper right')
    ax.legend_.remove()
    ax.set_xlabel('Time (min)')
    ax.set_ylabel(yaxis)

def getgermtime(df,germthresh=0,dormthresh=1500,dormassign=500):
    """function that returns the timepoint when the phase contrast timetrace first drops below a certain threshold (germthresh, default zero).
    It also assigns an arbitrarily large number (dormassign, default 500), for spores that stay dormant (do not darken beyond a certain threshold, dormthresh, default 1500). """
    
    idx = []
    for trace in df.columns:
        df2 =df.loc[:,trace]
        if df2.min()>dormthresh:
                idx.append(dormassign)
        else:
            idx.append((df2.values < germthresh).argmax())
    return idx

def plotCDF(df,label='_nolegend_',color='k',length=289,interval=1):
    """function that plots the cumulative density function (CDF) of a given dataframe. """
    bins = np.arange(0,length-1,interval)
    counts, bin_edges = np.histogram(df, bins=bins, range=[0,length])
    cdf = np.cumsum(counts)
    plt.plot(bin_edges[1:], cdf/len(df),alpha=1,label=label,color=color,lw=2.5)
    
def getCDF(df,length=289,interval=1):
    """function that returns the cumulative density function (CDF) of a given dataframe. """
    bins = np.arange(0,length,interval)
    counts, bin_edges = np.histogram(df, bins=bins, range=[0,length])
    cdf = np.cumsum(counts)
    return bin_edges[1:], cdf/len(df)

def getAvgCDF(germtimelist,length=289,interval=1):
    """function that returns the average CDF from several experiments. """
    cdflist = [getCDF(germtime,length,interval)[1] for germtime in germtimelist]
    avgcdf = np.mean(cdflist,axis=0)
    return avgcdf

def getStdCDF(germtimelist,length=289,interval=1):
    """function that returns the standard deviation of CDFs from several experiments. """
    cdflist = [getCDF(germtime,length,interval)[1] for germtime in germtimelist]
    stdcdf = np.std(cdflist,axis=0)
    return stdcdf

def plotAvgStdCDF(germtimelist,color,length=289,interval=1,label='_nolegend_'):
    """function that plots the average and standard deviation of CDFs from several experiments. """
    avgcdf = getAvgCDF(germtimelist,length)
    stdcdf = getStdCDF(germtimelist,length)
    binedges = np.arange(0,length-1,interval)
    plt.plot(binedges,1-avgcdf,alpha=0.7,label=label,color=color,lw=2)
    plt.fill_between(binedges,1-avgcdf+stdcdf,1-avgcdf-stdcdf,alpha=0.1,color=color)
    

# Read in phase contrast timetrace CSVs and calculate germination time

- Read in imageJ Results files using `readMultiMeasureData`
- Subtract background intensity using `subtractBackground`
- Calculate germination time ("germtime") from phase contrast traces using `getgermtime`
- Save germtime to CSV files

- The code below assumes ImageJ data for 2 experiments (`movie_1` and `movie_2`), each with a single-cell phase contrast timetrace file and a corresponding background intensity timetrace file.

In [14]:
movie_1_PHdf = readMultiMeasureData('/imageJ_results/movie_1_PH.csv') #single-cell phase contrast timetraces
movie_1_PHdf_bg = readMultiMeasureData('/imageJ_results/movie_1_PH_bg.csv') #background data
movie_1_PHdf = subtractBackground(movie_1_PHdf,movie_1_PHdf_bg) #subtract background
movie_1_germtime = getgermtime(movie_1_PHdf,germthresh=0) #calculate germination time for each spore
movie_1_germdf = pd.DataFrame(movie_1_germtime) #convert into Pandas dataframe
pd.DataFrame.to_csv(movie_1_germdf,dir+'/germination_time/movie_1_germtime.csv') #save as csv

movie_2_PHdf = readMultiMeasureData('/imageJ_results/movie_2_PH.csv')
movie_2_PHdf_bg = readMultiMeasureData('/imageJ_results/movie_2_PH_bg.csv')
movie_2_PHdf = subtractBackground(movie_2_PHdf,movie_2_PHdf_bg)
movie_2_germtime = getgermtime(movie_2_PHdf,germthresh=0) 
movie_2_germdf = pd.DataFrame(movie_2_germtime) 
pd.DataFrame.to_csv(movie_2_germdf,dir+'/germination_time/movie_2_germtime.csv') 

movie_PHdf = pd.concat([movie_1_PHdf,movie_2_PHdf],axis=1) #create a dataframe with all the data

Generally it is a good idea to check the phase contrast timetraces to see if the detection threshold (germthresh, default 0) works well. 

In [ ]:
plotTraces(movie_1_PHdf) #plot the first movie's timetraces
plt.axhline(y=0,color='k',ls='--') #plot a horizontal line at zero

# Read in saved germination times
- Read in saved germtimes as Pandas DataFrames.
- This saves time compared to calculating all of the germtimes each time you open the notebook, because reading in CSVs is much faster than calculating

In [ ]:
movie_1_germtime = pd.read_csv(dir+'/droptime/movie_1_germtime.csv').iloc[:,1]
movie_2_germtime = pd.read_csv(dir+'/droptime/movie_2_germtime.csv').iloc[:,1]

# Plot CDFs
Cumulative distribution functions (CDFs) are useful to visualize fractions vs time. Here we are comparing the germination time response across different strains, with the average and standard deviation from the different positions visualized accordingly.

In [ ]:
plotAvgStdCDF([movie_1_germtime,movie_2_germtime],color='blue')
labels= np.arange(0,28)
plt.xticks(np.arange(0,334,12),labels,size=10);
plt.xlabel('Time (hours)',size=15)
plt.ylabel('Dormant fraction',size=15)
plt.xlim(0,270);